# Evaulate Distilbert TopicChooser Model
### 25 Nov 2020

## I. Evaluation Parameters

In [1]:
# Set Evaulation Parameters
MODEL_NAME = "roberta-base"
MODEL_FILE ="saved_model_epoch10_20201127_2032.tar"
VAL_DATASET = "val_data_27Nov_gc_roberta.pickle"
EVAL_BATCH_SIZE = 64
DESCRIPTION = "Roberta Gossipcop model"

## II. Setup and Load Model

In [2]:
import pickle
import sys

import json
import pandas as pd
import pickle
import sklearn.metrics
import tqdm
import torch
import transformers as hft

sys.path.insert(0, "/home/jupyter")
import util.log
import util.data

In [3]:
# Load evaluation data
with open(VAL_DATASET, "rb") as vfile:
    val_dataset = pickle.load(vfile)

In [4]:
# Load saved model
checkpoint = torch.load(MODEL_FILE)
model = (hft.RobertaForSequenceClassification
         .from_pretrained(MODEL_NAME))
model.load_state_dict(checkpoint["model"])

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

<All keys matched successfully>

In [5]:
# Move model to GPU if available
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
model.to(device)
model.eval();
device

device(type='cuda')

## III. Evaluation Loop and Results Dataframe

In [6]:
# Evaulate model
eval_loader = torch.utils.data.DataLoader(val_dataset,
                                batch_size=EVAL_BATCH_SIZE,
                                shuffle=False)
labels = []
preds = []
logits = []
probs = []
sources = []
article_tokens = []
file_names = []

softmax = torch.nn.Softmax(dim=1)

with torch.no_grad():
    for art_num, article in tqdm.tqdm(enumerate(eval_loader)):
        input_ids = article["input_ids"].to(device)
        attent_mask = article["attention_mask"].to(device)
        output = model(input_ids,
                       attention_mask=attent_mask,
                       output_hidden_states=False,
                       output_attentions=False)
        prob = softmax(output[0].detach().cpu()).numpy()
        logit = output[0].detach().cpu().numpy()
        pred = [0 if lgt[0] > lgt[1] else 1 for lgt in logit]
        label = article["labels"].numpy()
        labels.extend(label)
        preds.extend(pred)
        logits.extend(logit)
        probs.extend(prob)
        article_tokens.extend(article["article_tokens"].numpy())
        sources.extend(article["sources"])
        file_names.extend(article["file_names"])

56it [01:00,  1.08s/it]


In [7]:
# Label of 0 is real, 1 is fake
eval_results = pd.DataFrame({"Article": list(range(len(labels))),
                             "Source": sources,
                             "Token_Length": article_tokens,
                             "Predictions": preds,
                             "Labels": labels,
                             "Logit_real(0)": [x[0] for x in logits],
                             "Logit_fake(1)": [x[1] for x in logits],
                             "Prob_real(0)": [x[0] for x in probs],
                             "Prob_real(1)": [x[1] for x in probs],
                             "File_Name": file_names})

eval_results

,Article,Source,Token_Length,Predictions,Labels,Logit_real(0),Logit_fake(1),Prob_real(0),Prob_real(1),File_Name
0,0,gossipcop,194,0,0,2.449035,-2.555168,0.993335,0.006665,gossipcop-860374
1,1,gossipcop,436,0,0,2.418322,-2.563293,0.993184,0.006816,gossipcop-907444
2,2,gossipcop,478,0,0,2.433471,-2.571512,0.993340,0.006660,gossipcop-871979
3,3,gossipcop,183,1,1,-0.183416,0.272883,0.387864,0.612136,gossipcop-9746788878
4,4,gossipcop,399,0,0,2.451972,-2.592970,0.993599,0.006401,gossipcop-846726
...,...,...,...,...,...,...,...,...,...,...
3523,3523,gossipcop,734,0,0,2.429535,-2.544417,0.993132,0.006868,gossipcop-881564
3524,3524,gossipcop,1192,0,0,2.448694,-2.575432,0.993466,0.006534,gossipcop-888811
3525,3525,gossipcop,634,0,0,2.439305,-2.570869,0.993374,0.006626,gossipcop-905426
3526,3526,gossipcop,166,0,1,0.952438,-0.963551,0.871690,0.128310,gossipcop-1120081991


In [8]:
# Save evaluation data to csv file
eval_results.to_csv("eval_data_" + MODEL_FILE[12:-4] + ".csv")

## IV. Overall Metrics

In [9]:
# Overall metrics
precision, recall, f1, _ = sklearn.metrics.precision_recall_fscore_support(
    eval_results.Labels, eval_results.Predictions, average="binary")
accuracy = sklearn.metrics.accuracy_score(eval_results.Labels, eval_results.Predictions)

metrics = {"description": DESCRIPTION,
           "eval_notes": "Evaluating both gossipcop and poltifact data",
           "model": MODEL_FILE,
           "epochs": checkpoint["epoch"],
           "batch_size": EVAL_BATCH_SIZE,
           "Date_evaluated": "25Nov2020",
           "train_data": "train_dataset_25Nov_source_distilbert.pickle",
           "val_data": "eval_data_" + MODEL_FILE[12:-4] + ".csv",
           "eval_data": VAL_DATASET,
           "accuracy": accuracy,
           "precision": precision,
           "recall": recall,
           "f1": f1}
metrics

{'description': 'Roberta Gossipcop model',
 'eval_notes': 'Evaluating both gossipcop and poltifact data',
 'model': 'saved_model_epoch10_20201127_2032.tar',
 'epochs': 10,
 'batch_size': 64,
 'Date_evaluated': '25Nov2020',
 'train_data': 'train_dataset_25Nov_source_distilbert.pickle',
 'val_data': 'eval_data_epoch10_20201127_2032.csv',
 'eval_data': 'val_data_27Nov_gc_roberta.pickle',
 'accuracy': 0.8622448979591837,
 'precision': 0.7454308093994778,
 'recall': 0.6624129930394431,
 'f1': 0.7014742014742015}

In [10]:
# Save metrics to JSON text file.
with open("eval_metrics_" + MODEL_FILE[12:-4] + ".json", "wt") as jfile:
    json.dump(metrics, jfile)

In [11]:
cm = pd.DataFrame(sklearn.metrics.confusion_matrix(eval_results.Labels, eval_results.Predictions))
label_titles = {0: "real", 1: "fake"}
cm.rename(index=label_titles, columns=label_titles, inplace=True)
cm

,real,fake
real,2471,195
fake,291,571
